In [1]:
import os, re, json
from collections import Counter

In [2]:
data_dir = os.path.dirname("./data/")
talks_data = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if '.json' in f]

talks = []
for filepath in talks_data:
    with open(filepath, 'r') as f:
        try:
            talks.append(json.load(f))
        except json.JSONDecodeError:
            continue

In [3]:
len(talks_data), len(talks)

(169163, 96542)

In [4]:
Counter([talk['category'].split(';')[0] for talk in talks]).most_common()

[('엽기&호러', 17501),
 ('엔터톡', 16834),
 ('웃기는 사진/동영상', 6733),
 ('결혼/시집/친정', 6290),
 ('10대 이야기', 5466),
 ('동물 사랑방', 5303),
 ('해석 남/여', 4269),
 ('여자들끼리만', 3604),
 ('20대 이야기', 2795),
 ('지금은 연애중', 2697),
 ('배꼽조심 유머', 2668),
 ('사는 얘기', 2600),
 ('사랑, 고백해도 될까?', 2304),
 ('헤어진 다음날', 2261),
 ('개념상실한사람들', 1768),
 ('남편 vs 아내', 1669),
 ('사랑과 이별', 1651),
 ('요리&레시피', 1481),
 ('회사생활', 1076),
 ('세상에이런일이', 1017),
 ('훈훈한 이야기', 947),
 ('나억울해요', 835),
 ('건강/다이어트', 822),
 ('남자들끼리만', 629),
 ('임신/출산/육아', 619),
 ('뷰티&스타일', 583),
 ('여행을 떠나요', 350),
 ('스타/스포츠', 267),
 ('30대 이야기', 222),
 ('TV톡', 213),
 ('싱글톡', 181),
 ('대한민국 이슈', 156),
 ('판춘문예', 135),
 ('군화와 고무신', 121),
 ('취업과 면접', 89),
 ('알바 경험담', 67),
 ('맞벌이 부부 이야기', 39),
 ('컬투쇼', 38),
 ('군대일기', 38),
 ('백수&백조 이야기', 33),
 ('믿음과신앙', 32),
 ('묻고 답하기', 31),
 ('남자들의 속깊은 이야기', 27),
 ('이슈토론', 13),
 ('패션왕코리아', 12),
 ('포토 스토리', 9),
 ('웹툰&카툰', 8),
 ('40대 이야기', 6),
 ('로맨스가 더 필요해', 6),
 ('코미디빅리그', 5),
 ('음담패설', 4),
 ('야구야구', 3),
 ('언스타일', 3),
 ('직장여성의 애환', 3),
 ('방송의 적', 2),
 

In [3]:
gsc = [talk for talk in talks if talk['category'].split(';')[0] == '결혼/시집/친정']
gsc_text = [talk['text'] for talk in gsc]
ent_text = [talk['text'] for talk in talks if talk['category'].split(';')[0] == '엔터톡']
yupgi_text = [talk['text'] for talk in talks if talk['category'].split(';')[0] == '엽기&호러']
teens_text = [talk['text'] for talk in talks if talk['category'].split(';')[0] == '10대 이야기']
twenties_text = [talk['text'] for talk in talks if talk['category'].split(';')[0] == '20대 이야기']

len(yupgi_text), len(ent_text), len(gsc_text), len(teens_text), len(twenties_text)

(22892, 26554, 12742, 10898, 4715)

In [10]:
flatten = lambda l: [item for sublist in l for item in sublist]

gsc_dump = flatten(gsc_text)
ent_dump = flatten(ent_text)
yupgi_dump = flatten(yupgi_text)
teens_dump = flatten(teens_text)
twenties_dump = flatten(twenties_text)

In [29]:
len(set(gsc_dump)),len(set(ent_dump)),len(set(yupgi_dump)),len(set(teens_dump)),len(set(twenties_dump))

# not sure if character-based approach, like https://github.com/karpathy/char-rnn,
# would be a good idea because of very high dimension (100x that of english)

(3074, 3262, 4369, 3001, 2250)

In [6]:
# markov.py by Allison Parrish
# http://www.decontextualize.com/teaching/rwet/n-grams-and-markov-chains/

import random

def build_model(tokens, n):
	"Builds a Markov model from the list of tokens, using n-grams of length n."
	model = dict()
	if len(tokens) < n:
		return model
	for i in range(len(tokens) - n):
		gram = tuple(tokens[i:i+n])
		next_token = tokens[i+n]
		if gram in model:
			model[gram].append(next_token)
		else:
			model[gram] = [next_token]
	final_gram = tuple(tokens[len(tokens)-n:])
	if final_gram in model:
		model[final_gram].append(None)
	else:
		model[final_gram] = [None]
	return model

def generate(model, n, seed=None, max_iterations=100):
	"""Generates a list of tokens from information in model, using n as the
		length of n-grams in the model. Starts the generation with the n-gram
		given as seed. If more than max_iteration iterations are reached, the
		process is stopped. (This is to prevent infinite loops)""" 
	if seed is None:
		seed = random.choice(list(model)) # python 3 doesn't support indexing of dict keys. change to list
	output = list(seed)
	current = tuple(seed)
	for i in range(max_iterations):
		if current in model:
			possible_next_tokens = model[current]
			next_token = random.choice(possible_next_tokens)
			if next_token is None: break
			output.append(next_token)
			current = tuple(output[-n:])
		else:
			break
	return output

def merge_models(models):
	"Merges two or more Markov models."
	merged_model = dict()
	for model in models:
		for key, val in model.iteritems():
			if key in merged_model:
				merged_model[key].extend(val)
			else:
				merged_model[key] = val
	return merged_model

def generate_from_token_lists(token_lines, n, count=14, max_iterations=100):
	"""Generates text from a list of lists of tokens. This function is intended
		for input text where each line forms a distinct unit (e.g., poetry), and
		where the desired output is to recreate lines in that form. It does this
		by keeping track of the n-gram that comes at the beginning of each line,
		and then only generating lines that begin with one of these "beginnings."
		It also builds a separate Markov model for each line, and then merges
		those models together, to ensure that lines end with n-grams statistically
		likely to end lines in the original text.""" 
	beginnings = list()
	models = list()
	for token_line in token_lines:
		beginning = token_line[:n]
		beginnings.append(beginning)
		line_model = build_model(token_line, n)
		models.append(line_model)
	combined_model = merge_models(models)
	generated_list = list()
	for i in range(count):
		generated_str = generate(combined_model, n, random.choice(beginnings),
				max_iterations)	
		generated_list.append(generated_str)
	return generated_list

def char_level_generate(lines, n, count=14, max_iterations=100):
	"""Generates Markov chain text from the given lines, using character-level
		n-grams of length n. Returns a list of count items."""
	token_lines = [list(line) for line in lines]
	generated = generate_from_token_lists(token_lines, n, count, max_iterations)
	return [''.join(item) for item in generated]

def word_level_generate(lines, n, count=14, max_iterations=100):
	"""Generates Markov chain text from the given lines, using word-level
		n-grams of length n. Returns a list of count items."""
	token_lines = [line.split() for line in lines]
	generated = generate_from_token_lists(token_lines, n, count, max_iterations)
	return [' '.join(item) for item in generated]

In [11]:
gsc_4 = build_model(gsc_dump, 4)
ent_4 = build_model(ent_dump, 4)
yupgi_4 = build_model(yupgi_dump, 4)
teens_4 = build_model(teens_dump, 4)
twenties_4 = build_model(twenties_dump, 4)

In [26]:
# gsc_gen = generate(gsc_4, 4, max_iterations=280)
# ent_gen = generate(ent_4, 4, max_iterations=280)
# yupgi_gen = generate(yupgi_4, 4, max_iterations=280)
# teens_gen = generate(teens_4, 4, max_iterations=280)
# twenties_gen = generate(twenties_4, 4, max_iterations=280)

print('네이트판 명예의 전당에 올라간 글을 재료로 텍스트를 생성하는 프로젝트를 시작했습니다.')
print('우선 어떻게 되는지 보기 위해 유니코드 문자 단위로 n-gram 마코프 모델을 만들어 텍스트를 만들어봤습니다.')
print('코드는 앨리슨 패리시 선생님의 수업 예제를 긁어왔습니다.')
print('출처: http://www.decontextualize.com/teaching/rwet/n-grams-and-markov-chains/')
print('n 값은 4입니다.')
print('카테고리마다 말투나 소재가 되는 단어 등이 다른 것을 어렴풋이 확인할 수 있습니다. 아직 제대로 된 문장이 나오지는 않습니다.')
print('\n=====결혼/시집/친정=====')
print(''.join(gsc_gen))
print('\n========엔터톡========')
print(''.join(ent_gen))
print('\n=======엽기&호러=======')
print(''.join(yupgi_gen))
print('\n======10대 이야기======')
print(''.join(teens_gen))
print('\n======20대 이야기======')
print(''.join(twenties_gen))
print('\n=====================')
print('')
print('다음 단계로는 자료를 신경망에 집어넣을 경우 더 그럴싸한 문장을 생성할 수 있는지 실험해보려 합니다.')
print('안드레이 카파시의 유명한 프로젝트를 참고할 예정입니다: https://github.com/karpathy/char-rnn')
print('다만 영문 알파벳보다 한국어의 feature dimension이 훨씬 클 테므로')
print('어떻게 효과적인 학습이 가능할지는 더 알아봐야 할 듯합니다.')
print('김태훈 님의 한국어 시 생성 프로젝트도 좀 더 자세히 보려고 합니다: https://github.com/carpedm20/poet-neural')
print('또한 그냥 문자열을 쓰지 않고 konlpy 패키지 등을 이용해 형태소로 변환한 자료를 사용하면 어떨지도 봐야겠습니다.')

네이트판 명예의 전당에 올라간 글을 재료로 텍스트를 생성하는 프로젝트를 시작했습니다.
우선 어떻게 되는지 보기 위해 유니코드 문자 단위로 n-gram 마코프 모델을 만들어 텍스트를 만들어봤습니다.
코드는 앨리슨 패리시 선생님의 수업 예제를 긁어왔습니다.
출처: http://www.decontextualize.com/teaching/rwet/n-grams-and-markov-chains/
n 값은 4입니다.
카테고리마다 말투나 소재가 되는 단어 등이 다른 것을 어렴풋이 확인할 수 있습니다. 아직 제대로 된 문장이 나오지는 않습니다.

=====결혼/시집/친정=====
걸로 민감해지네요..시어머님 도와드리는데, 그러다 아이한테 점까지도 제사 어머님께 화낸적도 없는 빈공간있는데, 저는 하루도안빼고 마사지도 3달이 지났는데 그래도 아들얘기하지만 누가봐도 인정을 못받았어요 어머니가 아니었다고 함. 남자친구는 "어맛! 왜 저의 사정은 자주쓰는데 두시간쯤떨어진곳에 있었습니다.
아내의입장에선 말그대로 친정따로 가야하고, 실제로 남자친구있고요. 미안한 부분도 있었고.
 
이렇게 오랫동안 안가다가 할아버지 앞에 계시던 다른방에서 여자는 제가 가고, 스님처럼 쿨하게 넘어가면 옆으로 누

========엔터톡========
괜히 곤란해요ㅠㅠㅜㅜㅜㅜㅜㅜㅜㅜㅜ 또 노래를 조카 잘생기고 멋지네
의외의 창섭이는 자세도 하나도 없었고
미모도 상큼하면서 본 지평선은 반듯반듯한 이정재야...응... 우린 그럴수 있죠 
아파하는줄알아... 이름 겹치는 바람 맞는 거 보기 좋은 것 같다......!!
제가 알아서 
공중부양하면서 
시크한모습 너무 안타까웠던 루나는 뮤지션은 랩퍼들은 뺌
 
 
 
 
 
 
 
http://fimg2.pann.com/new/download.jsp?FileID=33324]
http://fimg2.pann.co

=======엽기&호러=======
를
휘감고 사는 남자를 꺼낸 적이 있어서 뒤를 돌아보았으나 조용한 기분이 나빠졌는지 어떤 여자가 갑자기 목걸이에서 죽었다고

In [6]:
from konlpy.tag import Twitter
# from konlpy.tag import Hannanum, Kkma, Komoran, Mecab
from konlpy.utils import pprint

In [28]:
# hannanum = Hannanum() # divides up morphemes too much
# kkma = Kkma() # slow
# komoran = Komoran() # memory error 
# mecab = Mecab() # install fail
twitter = Twitter() # seems to be currently best for text generation
pprint(kkma.sentences(gsc[0]['text']))
print(' '.join(twitter.morphs(gsc_text[0])))
# morphemes ignore whitespace. can I featurize whitespace with konlpy?
# or, can the morphemes be combined back to sentences with whitespace?

In [134]:
# twitter_morphs = []
# for text in gsc_text:
#     twitter_morphs += twitter.morphs(text)
# len(twitter_morphs), len(set(twitter_morphs))

(4169627, 57122)